# VESC Real-Time Data Visualization

This notebook lets you see VESC data as **live, updating graphs**! Each cell below will show a different measurement from the VESC, updating in real-time as you spin the motor or change settings.

## How to Use This Notebook

1. **Run the setup cell first** to connect to the VESC
2. **Choose a graph to watch** by running one of the cells below
3. **Spin the motor by hand** or use motor commands to see the data change
4. **Press Ctrl+C** or interrupt the kernel to stop a graph

💡 **Tip**: Try running a graph, then manually spinning the motor to see real-time changes!

## Setup - Connect to VESC

In [ ]:
# Setup Python path to find student_api from parent directory
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from student_api import VESCStudentAPI
import time
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# Create API instance
vesc_api = VESCStudentAPI()

# Start the VESC system
if vesc_api.start():
    print("VESC system started successfully!")
    
    # Get controller for VESC ID 74 (as specified in README)
    vesc = vesc_api.get_controller(74)
    
    if vesc:
        print("Connected to VESC controller!")
    else:
        print("Failed to get VESC controller")
else:
    print("Failed to start VESC system")

## How to Use This Notebook

1. **Run the setup cell first** to connect to the VESC
2. **Choose a graph to watch** by running one of the cells below
3. **Manually spin the motor by hand** to see the data change in real-time
4. **Press Ctrl+C** or interrupt the kernel to stop a graph

💡 **Tip**: Try running a graph, then manually spinning the motor at different speeds to see how the data responds!

## Real-Time Graph Function

This helper function creates live updating graphs:

In [ ]:
def plot_realtime_data(get_data_func, title, ylabel, color='blue', duration=30, update_rate=10):
    """
    Create a real-time updating plot
    
    Args:
        get_data_func: Function to get data (like vesc.get_rpm)
        title: Graph title
        ylabel: Y-axis label
        color: Line color
        duration: How long to run (seconds)
        update_rate: Updates per second
    """
    # Setup data storage
    max_points = duration * update_rate
    times = deque(maxlen=max_points)
    values = deque(maxlen=max_points)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 6))
    line, = ax.plot([], [], color=color, linewidth=2)
    ax.set_title(f'{title} - Real-Time Data', fontsize=16, fontweight='bold')
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True, alpha=0.3)
    
    start_time = time.time()
    
    print(f"📊 Starting {title} visualization...")
    print(f"⏱️ Will run for {duration} seconds at {update_rate}Hz")
    print(f"🛑 Press Ctrl+C or interrupt kernel to stop early\n")
    
    try:
        while time.time() - start_time < duration:
            current_time = time.time() - start_time
            
            # Get new data point
            try:
                value = get_data_func()
                if value is None:
                    value = 0
            except:
                value = 0
            
            # Store data
            times.append(current_time)
            values.append(value)
            
            # Update plot
            if len(times) > 1:
                line.set_data(list(times), list(values))
                
                # Auto-scale axes
                ax.set_xlim(max(0, current_time - 20), current_time + 1)
                if values:
                    val_min, val_max = min(values), max(values)
                    margin = abs(val_max - val_min) * 0.1 + 0.1
                    ax.set_ylim(val_min - margin, val_max + margin)
                
                # Show current value
                ax.set_title(f'{title} - Current: {value:.2f} {ylabel.split()[0]}', 
                           fontsize=16, fontweight='bold')
                
                plt.draw()
                plt.pause(1/update_rate)
            
            time.sleep(max(0, 1/update_rate - 0.01))  # Maintain update rate
    
    except KeyboardInterrupt:
        print("\n🛑 Visualization stopped by user")
    
    print(f"✅ {title} visualization complete!")
    print(f"📈 Collected {len(values)} data points")
    if values:
        print(f"📊 Final value: {values[-1]:.2f} {ylabel.split()[0]}")
        print(f"📊 Min: {min(values):.2f}, Max: {max(values):.2f}, Avg: {np.mean(values):.2f}")
    
    plt.show()

## Motor Speed (RPM) - Real-Time Graph

Watch the motor speed change in real-time! Perfect for seeing how fast the motor spins.

💡 **Try this**: Run the cell, then spin the motor by hand to see the RPM change!

In [ ]:
# Real-time RPM visualization
plot_realtime_data(
    get_data_func=vesc.get_rpm,
    title="Motor Speed (RPM)",
    ylabel="RPM (Revolutions Per Minute)",
    color='red',
    duration=30,
    update_rate=10
)

## Duty Cycle - Real-Time Graph

Watch the throttle setting! This shows how much power the VESC is sending to the motor.

💡 **Note**: This will normally stay at 0 since we're not sending motor commands - it only changes when the VESC receives control signals.

In [ ]:
# Real-time Motor Current visualization
plot_realtime_data(
    get_data_func=vesc.get_motor_current,
    title="Motor Current",
    ylabel="Current (Amperes)",
    color='orange',
    duration=30,
    update_rate=10
)

## Duty Cycle - Real-Time Graph

Watch the throttle setting! This shows how much power the VESC is sending to the motor.

💡 **Try this**: Send motor commands in another cell to see this change!

In [ ]:
# Real-time Duty Cycle visualization
plot_realtime_data(
    get_data_func=vesc.get_duty_cycle,
    title="Duty Cycle (Throttle)",
    ylabel="Duty Cycle (-1.0 to 1.0)",
    color='purple',
    duration=30,
    update_rate=10
)

## Battery Voltage - Real-Time Graph

Monitor your battery level! This shows the voltage of your battery in real-time.

💡 **This should always show a value** - it's your battery voltage!

In [ ]:
# Real-time Battery Voltage visualization
plot_realtime_data(
    get_data_func=vesc.get_input_voltage,
    title="Battery Voltage",
    ylabel="Voltage (Volts)",
    color='green',
    duration=30,
    update_rate=5  # Slower update rate since voltage changes slowly
)

## Battery Current - Real-Time Graph

See how much electricity is being drawn from the battery! This is different from motor current.

💡 **Try this**: Compare this to motor current - they're often different!

In [ ]:
# Real-time Battery Current visualization
plot_realtime_data(
    get_data_func=vesc.get_input_current,
    title="Battery Current",
    ylabel="Current (Amperes)",
    color='blue',
    duration=30,
    update_rate=10
)

## Controller Temperature - Real-Time Graph

Watch the VESC controller heat up! Important for monitoring system health.

💡 **Safety**: If temperature goes above 80°C, give the controller a break!

In [ ]:
# Real-time Controller Temperature visualization
plot_realtime_data(
    get_data_func=vesc.get_fet_temperature,
    title="Controller Temperature (FET)",
    ylabel="Temperature (°C)",
    color='red',
    duration=30,
    update_rate=5  # Slower update - temperature changes slowly
)

## Motor Temperature - Real-Time Graph

Monitor motor heat! Motors can get hot when working hard.

💡 **Safety**: If motor temperature goes above 100°C, let it cool down!

In [ ]:
# Real-time Motor Temperature visualization
plot_realtime_data(
    get_data_func=vesc.get_motor_temperature,
    title="Motor Temperature",
    ylabel="Temperature (°C)",
    color='darkred',
    duration=30,
    update_rate=5  # Slower update - temperature changes slowly
)

## Multiple Graphs at Once

Want to see multiple measurements at the same time? This cell shows 4 key measurements together!

💡 **Try this**: Run this cell, then manually spin the motor to see how all the values change together!

## Manual Testing Guide

Want to see interesting data in your graphs? Here are safe ways to generate data by manually spinning the motor:

### 🖐️ **Manual Motor Testing (Safe)**

**1. 🔄 SPIN SLOWLY by hand:**
- Watch RPM stay low (under 100)
- See small current values
- Tachometer counts up slowly

**2. 🌪️ SPIN FAST by hand:**
- Watch RPM jump higher
- See current increase
- Tachometer counts up quickly

**3. 🛑 SPIN THEN STOP suddenly:**
- Watch values spike then drop to zero
- See the decay curves

**4. ↔️ SPIN BOTH DIRECTIONS:**
- Forward spin = positive RPM
- Backward spin = negative RPM
- Current can go both ways too

**5. 🔋 WATCH VOLTAGE:**
- Should stay steady around your battery voltage
- May drop slightly under heavy spin

💡 **Best Results**: Start a graph, then manually spin the motor!
🔒 **This is much safer** than using motor control commands.

# Example: Test the graphs with manual spinning

print(\"🎯 How to Test Your Graphs:\")\nprint()\nprint(\"1. 📊 Start any graph above\")\nprint(\"2. 🖐️ Manually spin the motor while graph is running\")\nprint(\"3. 👀 Watch the real-time data change!\")\nprint()\nprint(\"🔄 Try different spinning techniques:\")\nprint(\"   • Slow steady spin\")\nprint(\"   • Fast spin burst\")\nprint(\"   • Spin forward then backward\")\nprint(\"   • Spin fast then stop suddenly\")\nprint()\nprint(\"💡 This is the safest way to see live VESC data!\")

## Tips for Using Real-Time Graphs

### 🎯 **Getting Good Data:**
1. **Manually spin the motor by hand** to see RPM and current changes
2. **Try different spin speeds** - slow vs fast
3. **Spin in both directions** to see positive and negative values
4. **Watch voltage** - it should always show your battery level
5. **Monitor temperatures** during extended testing

### 📊 **Understanding the Graphs:**
- **Flat lines at zero** = Normal when motor isn't moving
- **Spiky data** = Normal - motors create electrical noise when spinning
- **Voltage steady** = Battery level (should stay fairly constant)
- **Tachometer increasing** = Counts every rotation (like an odometer)

### 🔧 **Troubleshooting:**
- **Graph not updating?** Check VESC connection
- **All zeros?** Try spinning motor manually
- **Graph too fast/slow?** Adjust `update_rate` parameter in the code
- **Want longer recording?** Increase `duration` parameter in the code

### 🚀 **Fun Experiments:**
- Watch how RPM and current relate when you spin at different speeds
- See temperature changes during extended manual testing
- Compare motor current vs battery current
- Watch the tachometer count up as you spin
- Try spinning fast then stopping - see the decay curves!

### 🔒 **Safety Note:**
Manual spinning is much safer than motor control commands. You have complete control over the speed and can stop instantly. This is the recommended way to generate interesting data for the graphs!

Have fun visualizing your VESC data! 📈⚡

## Power Consumed (Watt Hours) - Real-Time Graph

More accurate energy measurement! Watt hours account for both current and voltage.

💡 **This is like your electricity bill** - measured in watt hours!

In [ ]:
# Real-time Power Consumed visualization
plot_realtime_data(
    get_data_func=vesc.get_watt_hours_consumed,
    title="Power Consumed",
    ylabel="Watt Hours (Wh)",
    color='navy',
    duration=30,
    update_rate=5
)

## Power Recovered (Watt Hours) - Real-Time Graph

See the power you recover from regenerative braking! This is the most accurate measure of energy recovery.

💡 **Compare this to power consumed** to see your efficiency!

In [ ]:
# Real-time Power Recovered visualization
plot_realtime_data(
    get_data_func=vesc.get_watt_hours_charged,
    title="Power Recovered (Regenerative Braking)",
    ylabel="Watt Hours (Wh)",
    color='forestgreen',
    duration=30,
    update_rate=5
)

## Total Rotations (Tachometer) - Real-Time Graph

Count every rotation the motor makes! Like an odometer for your motor.

💡 **This keeps counting up** every time the motor spins!

In [ ]:
# Real-time Tachometer visualization
plot_realtime_data(
    get_data_func=vesc.get_tachometer_value,
    title="Total Motor Rotations (Tachometer)",
    ylabel="Total Rotations",
    color='brown',
    duration=30,
    update_rate=10
)

## Multiple Graphs at Once

Want to see multiple measurements at the same time? This cell shows 4 key measurements together!

In [ ]:
def plot_multiple_realtime(duration=30, update_rate=5):
    """Show multiple VESC measurements in real-time"""
    
    # Setup data storage
    max_points = duration * update_rate
    times = deque(maxlen=max_points)
    rpm_data = deque(maxlen=max_points)
    current_data = deque(maxlen=max_points)
    voltage_data = deque(maxlen=max_points)
    temp_data = deque(maxlen=max_points)
    
    # Create subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Setup each subplot
    line1, = ax1.plot([], [], 'r-', linewidth=2)
    ax1.set_title('Motor Speed (RPM)', fontweight='bold')
    ax1.set_ylabel('RPM')
    ax1.grid(True, alpha=0.3)
    
    line2, = ax2.plot([], [], 'orange', linewidth=2)
    ax2.set_title('Motor Current', fontweight='bold')
    ax2.set_ylabel('Current (A)')
    ax2.grid(True, alpha=0.3)
    
    line3, = ax3.plot([], [], 'g-', linewidth=2)
    ax3.set_title('Battery Voltage', fontweight='bold')
    ax3.set_ylabel('Voltage (V)')
    ax3.set_xlabel('Time (s)')
    ax3.grid(True, alpha=0.3)
    
    line4, = ax4.plot([], [], 'red', linewidth=2)
    ax4.set_title('Controller Temperature', fontweight='bold')
    ax4.set_ylabel('Temperature (°C)')
    ax4.set_xlabel('Time (s)')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    start_time = time.time()
    print(f"📊 Starting multi-parameter visualization for {duration} seconds...")
    print(f"🛑 Press Ctrl+C to stop early\n")
    
    try:
        while time.time() - start_time < duration:
            current_time = time.time() - start_time
            
            # Get all data
            rpm = vesc.get_rpm() or 0
            current = vesc.get_motor_current() or 0
            voltage = vesc.get_input_voltage() or 0
            temp = vesc.get_fet_temperature() or 0
            
            # Store data
            times.append(current_time)
            rpm_data.append(rpm)
            current_data.append(current)
            voltage_data.append(voltage)
            temp_data.append(temp)
            
            # Update plots
            if len(times) > 1:
                time_list = list(times)
                
                # Update RPM
                line1.set_data(time_list, list(rpm_data))
                ax1.set_xlim(max(0, current_time - 20), current_time + 1)
                if rpm_data:
                    margin = max(abs(max(rpm_data)), abs(min(rpm_data))) * 0.1 + 10
                    ax1.set_ylim(min(rpm_data) - margin, max(rpm_data) + margin)
                ax1.set_title(f'Motor Speed: {rpm:.1f} RPM', fontweight='bold')
                
                # Update Current
                line2.set_data(time_list, list(current_data))
                ax2.set_xlim(max(0, current_time - 20), current_time + 1)
                if current_data:
                    margin = max(abs(max(current_data)), abs(min(current_data))) * 0.1 + 0.5
                    ax2.set_ylim(min(current_data) - margin, max(current_data) + margin)
                ax2.set_title(f'Motor Current: {current:.2f} A', fontweight='bold')
                
                # Update Voltage
                line3.set_data(time_list, list(voltage_data))
                ax3.set_xlim(max(0, current_time - 20), current_time + 1)
                if voltage_data:
                    v_min, v_max = min(voltage_data), max(voltage_data)
                    margin = (v_max - v_min) * 0.1 + 0.5
                    ax3.set_ylim(v_min - margin, v_max + margin)
                ax3.set_title(f'Battery Voltage: {voltage:.2f} V', fontweight='bold')
                
                # Update Temperature
                line4.set_data(time_list, list(temp_data))
                ax4.set_xlim(max(0, current_time - 20), current_time + 1)
                if temp_data:
                    t_min, t_max = min(temp_data), max(temp_data)
                    margin = (t_max - t_min) * 0.1 + 2
                    ax4.set_ylim(t_min - margin, t_max + margin)
                ax4.set_title(f'Controller Temp: {temp:.1f}°C', fontweight='bold')
                
                plt.draw()
                plt.pause(1/update_rate)
            
            time.sleep(max(0, 1/update_rate - 0.02))
    
    except KeyboardInterrupt:
        print("\n🛑 Multi-parameter visualization stopped")
    
    plt.show()
    print("✅ Multi-parameter visualization complete!")

# Run the multi-parameter visualization
plot_multiple_realtime(duration=30, update_rate=5)

## Motor Control Helper (Optional)

Want to test motor commands while watching the graphs? Use this cell to send safe motor commands.

⚠️ **Safety First**: Only use this when it's safe for the motor to move!

In [ ]:
# SAFETY: Only enable this when safe to move the motor
MOTOR_CONTROL_ENABLED = False

if MOTOR_CONTROL_ENABLED:
    print("🎮 Motor Control Helper")
    print("Use these commands while running graphs above:\n")
    
    # Small duty cycle test
    print("Setting 10% throttle for 3 seconds...")
    vesc.set_duty_cycle(0.1)
    time.sleep(3)
    
    print("Setting 20% throttle for 3 seconds...")
    vesc.set_duty_cycle(0.2)
    time.sleep(3)
    
    print("Stopping motor...")
    vesc.set_duty_cycle(0)
    
    print("✅ Motor test complete!")
    
else:
    print("🛑 Motor control disabled for safety")
    print("Set MOTOR_CONTROL_ENABLED = True to enable motor commands")
    print("\n📝 Available commands when enabled:")
    print("   vesc.set_duty_cycle(0.1)    # 10% throttle")
    print("   vesc.set_current(2.0)       # 2A current")
    print("   vesc.set_brake_current(1.0) # 1A braking")
    print("   vesc.set_duty_cycle(0)      # STOP")

## Tips for Using Real-Time Graphs

### 🎯 **Getting Good Data:**
1. **Spin the motor by hand** to see RPM and current changes
2. **Use motor commands** (if safe) to see controlled changes
3. **Watch voltage** - it should always show your battery level
4. **Monitor temperatures** during extended use

### 📊 **Understanding the Graphs:**
- **Flat lines at zero** = Normal when motor isn't moving
- **Spiky data** = Normal - motors create electrical noise
- **Voltage slowly decreasing** = Battery discharging
- **Temperature slowly increasing** = Components warming up

### 🔧 **Troubleshooting:**
- **Graph not updating?** Check VESC connection
- **All zeros?** Try spinning motor manually
- **Graph too fast/slow?** Adjust `update_rate` parameter
- **Want longer recording?** Increase `duration` parameter

### 🚀 **Advanced Ideas:**
- Compare motor current vs battery current
- Watch temperature rise during heavy use
- Monitor energy recovery during braking
- Track efficiency over time

Have fun visualizing your VESC data! 📈⚡